# Online Retail Dataset — Exploration des données

Objectif :
Explorer le dataset afin de comprendre sa structure, identifier les problèmes de qualité des données et préparer les étapes de nettoyage et d’analyse.

Source :
Online Retail Dataset (Kaggle)

Imports et configuration

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


## Chargement des données et aperçu général

In [10]:
# Chargement du dataset
df = pd.read_csv('../data/raw/online_retail.csv')
# Affichage des 10 premières lignes
df.head(10)




,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


## Nombre de lignes et colonnes

In [11]:
print("Lignes :", df.shape[0], "\nColonnes :", df.shape[1])

Lignes : 541909 
Colonnes : 8


Le dataset est volumineux avec plus de 540 000 enregistrements et 8 colonnes. C'est un bon volume pour une analyse commerciale fiable.

## Types de données par colonne

In [12]:
# Types de données par colonne
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


Les colonnes numériques (Quantity, UnitPrice, CustomerID) coexistent avec du texte (InvoiceNo, StockCode, Description, Country). On remarque déjà que CustomerID a beaucoup de valeurs nulles (406 829 non-null sur 541 909), ce qui indique des clients anonymes.

## Statistiques descriptives

In [13]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


Les quantités sont centrées autour de 9 unités en médiane. Les prix unitaires s'étalent largement (min ~0, max ~21k), suggérant un mix de produits très variés. Le CustomerID médian est 15k, mais avec une grande variabilité.

## Valeurs manquantes par colonne

In [14]:

df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Plus d'un quart des enregistrements (135k) n'ont pas de ClientID. Les descriptions sont presque complètes (1.5k manquantes seulement). Les autres colonnes sont bien remplies.

## Valeurs uniques par colonne

In [15]:
df.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

4 000+ produits différents et 38 pays représentés. Les 25 900 numéros de facture suggèrent ~25k transactions, mais on a 540k lignes : chaque facture contient plusieurs articles. Le nombre de dates uniques (23k) indique une couverture de plusieurs années.

## Observations et problèmes détectés

Voici les problèmes détectés dans le dataset :

- **CustomerID** : 135 080 transactions (25 % du dataset) n’ont pas d’identifiant client.  
- **Description** : 1 454 lignes manquent de description du produit (moins de 1 % des données).  
- **Quantités négatives** : certaines transactions ont des quantités négatives, ce qui correspond probablement à des retours.  
- **Prix ou montants négatifs** : quelques transactions ont un prix ou un montant total négatif, ce qui semble être des anomalies.  
- **Format de la date** : `InvoiceDate` est enregistré comme texte et devra être converti en format datetime pour les analyses temporelles.  
- **Répartition par pays** : le dataset contient plusieurs pays, mais la majorité des transactions provient du Royaume-Uni.

Vérification rapide des anomalies :

In [16]:

print(f"Quantités négatives : {(df['Quantity'] < 0).sum()}")
print(f"Prix <= 0 : {(df['UnitPrice'] <= 0).sum()}")
print(f"Top 5 pays : {df['Country'].value_counts().head()}")


Quantités négatives : 10624
Prix <= 0 : 2517
Top 5 pays : Country
United Kingdom    495478
Germany             9495
France              8557
EIRE                8196
Spain               2533
Name: count, dtype: int64
